# 3d search using Annoy

Annoy 
- https://github.com/spotify/annoy

In [1]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import pickle
import numpy as np
import sys
import os
sys.path.append(os.pardir)
from src.search.search import Search
%matplotlib inline 
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='****.****', api_key='****')

Using TensorFlow backend.
/Users/masayaohgushi/sensor_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
class_id_to_name_modelnet40 = {
      1: "airplane",
      2: "bookshelf",
      3: "chair",
      4: "desk",
      5: "glass_box",
      6: "laptop",
      7: "person",
      8: "range_hood",
      9: "stool",
      10: "tv_stand",
      11: "bathtub",
      12: "bottle",
      13: "cone",
      14: "door",
      15: "guitar",
      16: "mantel",
      17: "piano",
      18: "sink",
      19: "table",
      20: "vase",
      21: "bed",
      22: "bowl",
      23: "cup",
      24: "dresser",
      25: "keyboard",
      26: "monitor",
      27: "plant",
      28: "sofa",
      29: "tent",
      30: "wardrobe",
      31: "bench",
      32: "car",
      33: "curtain",
      34: "flower_pot",
      35: "lamp",
      36: "night_stand",
      37: "radio",
      38: "stairs",
      39: "toilet",
      40: "xbox",
}

# Load data and label

In [3]:
with open("../data/processed/features_test.pkl", "rb") as f:    
    feature_data = pickle.load(f)

with open("../data/processed/labels_test.pkl", "rb") as f:    
    label = pickle.load(f)    

In [4]:
label_name_list = []
for z in label:
    label_name_list.append(class_id_to_name_modelnet40[z])

# 3D visualize method

In [5]:
def voxel_scatter(np_vox):
   # initialize empty array
    vox_scat = np.zeros([0, 3], dtype=np.uint32)

    # itterate through x-dimensions
    for x in range(0, np_vox.shape[1]):
        # itterate through y-dimension
        for y in range(0, np_vox.shape[2]):
            # itterate through z-dimension
            for z in range(0, np_vox.shape[3]):
                # if voxel is dense add to scatter array
                if np_vox[0, x, y, z] == 1.0:
                    arr_tmp = np.zeros([1, 3], dtype=np.uint32)
                    arr_tmp[0, :] = (x, y, z)
                    vox_scat = np.concatenate((vox_scat, arr_tmp))
    return vox_scat

def visualize_data(visualize_data):
    fig2 = plt.figure(2, figsize=(8, 10))
    ax1 = fig2.add_subplot(1,1,1, projection='3d')
    ax1.scatter(visualize_data[:,0], visualize_data[:,1], visualize_data[:,2])
    plt.draw()
    


## Get the unique label index

In [6]:
u, indices = np.unique(label_name_list, return_index=True)

for index in indices:
    print(label_name_list[index])

bathtub
bed
bench
bookshelf
bottle
bowl
car
chair
cone
cup
curtain
desk
door
dresser
flower_pot
glass_box
guitar
keyboard
lamp
laptop
mantel
monitor
night_stand
person
piano
plant
radio
range_hood
sink
sofa
stairs
stool
tent
toilet
tv_stand
vase
wardrobe
xbox


# Search

- Prepare
 - prepare feature extract model
 - prepare annoy search model

- Feature extract
- Search

In [7]:
search_instance = Search(
    featture=512,
    model_name="../models/model_net_epoch_19_acc_0.96.hdf5",
    regist_annoy="../models/feature_model_net_epoch_19_acc_0.96_all_test_data.ann",
)

In [8]:
label_index = -5

search_index = indices[label_index] + 1000
label_name_list[indices[label_index]]

'toilet'

## Extract feature

In [9]:
feature_data.shape

(28404, 1, 32, 32, 32)

In [10]:
extract_features = search_instance.extract_feature(np.reshape(feature_data[search_index], 
                                                              (1, 1, 
                                                              feature_data[search_index].shape[1],
                                                              feature_data[search_index].shape[2],
                                                              feature_data[search_index].shape[3],)))

extract index:  0


In [11]:
search_result = search_instance.search(extract_features[0])

In [12]:
search_result

[1240, 1381, 515, 1225, 983, 1354, 1341, 1169, 1292, 292]

# Visualize

visualize by plotly.

- It is better visualize than matplotlib
- Interactive action
- If you use it, you have to regist.

## Search label list

In [13]:
convert_data_list = []

for index in search_result:
    print(label_name_list[index])
    convert_data_list.append(voxel_scatter(feature_data[index]))

toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet
toilet


## Search object

In [14]:
convert_data = voxel_scatter(feature_data[search_index])

trace = go.Mesh3d(x=convert_data[:,0], y=convert_data[:,1], z=convert_data[:,2], color='#FFB6C1',opacity=0.50)
py.iplot([trace])

## Search result

In [15]:
convert_data = convert_data_list[0]

trace = go.Mesh3d(x=convert_data[:,0], y=convert_data[:,1], z=convert_data[:,2], color='#FFB6C1',opacity=1.00)
py.iplot([trace])



In [16]:
convert_data = convert_data_list[-1]

trace = go.Mesh3d(x=convert_data[:,0], y=convert_data[:,1], z=convert_data[:,2], color='#FFB6C1',opacity=1.00)
py.iplot([trace])